# 第9回課題

注：DocTestの結果は、課題3の末尾にまとめて掲載している。

## 課題1

まずBankAccountクラスの実装について述べる。

最初にinit関数を用いてインスタンス作成時のパスワードと残高を初期化する。

次に引き出し(withdraw)と預入(deposit)を定義する。これはパスワードの合致判定をし、口座残高を(可能であれば)加減算するだけなので説明は省く。

最後に、問題のために、クラスとwithdrawに説明を書き、withdrawにはdoctest用の実行例をつける。テスト結果は課題3の末尾にあるように全て問題ない。


In [81]:
class BankAccount:
    """
    Attributes:
        __balance (float): The account balance.
        __password (str): The account password for authentication.

    Methods:
        withdraw(amount, password): Withdraws money if the password is correct and balance is sufficient.
        deposit(amount, password): Deposits money if the password is correct.
    """

    def __init__(self, initial_balance, password):
        """
        Initializes a new BankAccount instance with a given balance and password.

        Args:
            initial_balance (float): The initial balance of the account.
            password (str): The password for the account.
        """
        self.__balance = initial_balance
        self.__password = password

    def withdraw(self, amount, password):
        """
        Withdraws the money with specified amount from the argument if the password matched and sufficient funds exist.

        Args:
            amount (float): The amount of money to withdraw.
            password (str): The password for authentication.

        Returns:
            float: The new balance after withdrawal if successful, or a string error message.

        DocTest:
        >>> account = BankAccount(1000, '0000')
        >>> account.withdraw(500, '0000')
        500
        >>> account.withdraw(600, '0000')
        'Insufficient funds'
        >>> account.withdraw(100, '1111')
        'Incorrect password'
        """
        
        if password == self.__password:
            if self.__balance >= amount:
                self.__balance -= amount
                return self.__balance
            else:
                return "Insufficient funds"
        else:
            return "Incorrect password"

    def deposit(self, amount, password):

        if password == self.__password:
            self.__balance += amount
            return self.__balance
        else:
            return "Incorrect password"


In [82]:
help(BankAccount)

Help on class BankAccount in module __main__:

class BankAccount(builtins.object)
 |  BankAccount(initial_balance, password)
 |  
 |  Attributes:
 |      __balance (float): The account balance.
 |      __password (str): The account password for authentication.
 |  
 |  Methods:
 |      withdraw(amount, password): Withdraws money if the password is correct and balance is sufficient.
 |      deposit(amount, password): Deposits money if the password is correct.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, initial_balance, password)
 |      Initializes a new BankAccount instance with a given balance and password.
 |      
 |      Args:
 |          initial_balance (float): The initial balance of the account.
 |          password (str): The password for the account.
 |  
 |  deposit(self, amount, password)
 |  
 |  withdraw(self, amount, password)
 |      Withdraws the money with specified amount from the argument if the password matched and sufficient funds exist.
 |      
 |     

In [83]:
help(BankAccount.withdraw)

Help on function withdraw in module __main__:

withdraw(self, amount, password)
    Withdraws the money with specified amount from the argument if the password matched and sufficient funds exist.
    
    Args:
        amount (float): The amount of money to withdraw.
        password (str): The password for authentication.
    
    Returns:
        float: The new balance after withdrawal if successful, or a string error message.
    
    DocTest:
    >>> account = BankAccount(1000, '0000')
    >>> account.withdraw(500, '0000')
    500
    >>> account.withdraw(600, '0000')
    'Insufficient funds'
    >>> account.withdraw(100, '1111')
    'Incorrect password'



## 課題2

Observer, Subjectの概念は初耳だったのでメモを残しておく。

Observer:時間によって変化する値を監視し、それを追随するもの。また、値を加工して表示したりするもの。

Subject:Observerが監視する値を保持するもの。

利点：Observer-Subjectの構成でプログラムを作ることで、「値の監視」「値の加工・表示」を切り離すことができ、お互いの再利用性が高まって嬉しい。




コードの説明：

Observer, Subject, ClockTimer, ConsoleClockは資料をもとに作っているため、説明は省く。

問題に回答するため、ClockTimerの中にdoctestを作成した。doctestが失敗していないので、複数のobserverを作成しても問題ないことが分かる。


In [84]:
import abc

class Observer(abc.ABC):
    """Abstract observer class."""
    @abc.abstractmethod
    def update(self):
        """Update status."""
        pass

class Subject:
    """Abstract subject class to manage observers."""
    def __init__(self):
        self.observers = []

    def attach(self, observer):
        """Add observer to the list."""
        self.observers.append(observer)

    def detach(self, observer):
        """Remove observer from the list."""
        self.observers.remove(observer)

    def notify(self):
        """Call update() for all observers."""
        for observer in self.observers:
            observer.update()

class ClockTimer(Subject):
    """
    Doctest:
    
    >>> clock_timer = ClockTimer(23, 59, 58)
    >>> observer1 = ConsoleClock(clock_timer)
    >>> observer2 = ConsoleClock(clock_timer)
    >>> clock_timer.tick() 
    23:59:59
    23:59:59
    >>> clock_timer.tick() 
    00:00:00
    00:00:00
    """
    def __init__(self, hour, minute, second):
        super().__init__()
        self.hour = hour
        self.minute = minute
        self.second = second

    def tick(self):
        """Advance the clock by one second and notify observers if time changes."""
        self.second += 1
        if self.second == 60:
            self.second = 0
            self.minute += 1
        if self.minute == 60:
            self.minute = 0
            self.hour += 1
        if self.hour == 24:
            self.hour = 0
        self.notify()

class ConsoleClock(Observer):
    """Display time in the console."""
    def __init__(self, subject):
        """create ConsoleClock and attach to subject"""
        super().__init__()
        self.subject = subject
        self.subject.attach(self)

    def clock_to_string(self, clock):
        return "{:02d}:{:02d}:{:02d}".format(clock.hour, clock.minute, clock.second)

    def update(self):
        print(self.clock_to_string(self.subject))

In [85]:
timer = ClockTimer(22, 54, 12)
timer.tick() 

In [86]:
cc = ConsoleClock(timer) 
timer.tick() 
timer.tick() 

22:54:14
22:54:15


In [87]:
timer.detach(cc)
timer.tick() 

## 課題3

Observerを継承させてCountdownClockを作る。ConsoleClockとほぼ同じだが、いくつか相違点がある。

・残り時刻を管理するため、カウントダウンのカウントsecondsを引数に持つ

・updateメソッド内でカウントダウンを行う

ここで、問題の指定通りclocktimerを変更せずtimer.tick()と書いているが、これは内部の変数を何も参照していないので、単にnotifyからCountdownClockのupdateメソッドを呼びたいだけである。

In [88]:
class CountdownClock(Observer):
    """
    Doctest:
    >>> timer = ClockTimer(0, 0, 0) 
    >>> cd = CountdownClock(timer, 3) 
    >>> timer.tick() 
    3
    >>> timer.tick()
    2
    >>> timer.tick()
    1
    >>> timer.tick()
    !!!
    """
    def __init__(self, subject, seconds):
        super().__init__()
        self.subject = subject
        self.seconds = seconds
        self.subject.attach(self)

    def update(self):

        if self.seconds > 0:
            self.seconds -= 1
            print(self.seconds + 1) 
        else:
            print("!!!")
            self.subject.detach(self)
        

In [89]:
# テストケース
timer=ClockTimer(0,0 ,0)
cd = CountdownClock(timer, 3) 
timer.tick()
timer.tick()
timer.tick()
timer.tick() 

3
2
1
!!!


課題1～3までのDoctestの結果をまとめて以下に示す。

In [90]:
doctest.testmod()

TestResults(failed=0, attempted=15)

In [91]:
import doctest
doctest.testmod(verbose=True)

Trying:
    account = BankAccount(1000, '0000')
Expecting nothing
ok
Trying:
    account.withdraw(500, '0000')
Expecting:
    500
ok
Trying:
    account.withdraw(600, '0000')
Expecting:
    'Insufficient funds'
ok
Trying:
    account.withdraw(100, '1111')
Expecting:
    'Incorrect password'
ok
Trying:
    clock_timer = ClockTimer(23, 59, 58)
Expecting nothing
ok
Trying:
    observer1 = ConsoleClock(clock_timer)
Expecting nothing
ok
Trying:
    observer2 = ConsoleClock(clock_timer)
Expecting nothing
ok
Trying:
    clock_timer.tick() 
Expecting:
    23:59:59
    23:59:59
ok
Trying:
    clock_timer.tick() 
Expecting:
    00:00:00
    00:00:00
ok
Trying:
    timer = ClockTimer(0, 0, 0) 
Expecting nothing
ok
Trying:
    cd = CountdownClock(timer, 3) 
Expecting nothing
ok
Trying:
    timer.tick() 
Expecting:
    3
ok
Trying:
    timer.tick()
Expecting:
    2
ok
Trying:
    timer.tick()
Expecting:
    1
ok
Trying:
    timer.tick()
Expecting:
    !!!
ok
19 items had no tests:
    __main__
    

TestResults(failed=0, attempted=15)